In [ ]:
# epub file, using ChatGLM (GPU) + trans-opus-mt-en-zh (CPU)
# trans-opus-mt-en-zh is use for translate short content

In [1]:
from langchain.document_loaders import UnstructuredEPubLoader
loader = UnstructuredEPubLoader("Your_file_here.epub", mode="elements", strategy="fast")
data = loader.load()

In [ ]:
# Load translate module
from transformers import AutoModelWithLMHead,AutoTokenizer,pipeline
mode_name = 'liam168/trans-opus-mt-en-zh'
model = AutoModelWithLMHead.from_pretrained(mode_name)
tokenizer = AutoTokenizer.from_pretrained(mode_name)
translation = pipeline("translation_en_to_zh", model=model, tokenizer=tokenizer, device="cpu")

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True).half().cuda()
model = model.eval()

In [7]:
from langchain.prompts import PromptTemplate
template = """
翻译成中文:\n
{question}
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
p2 = IntProgress(max=len(data))
p2.description = 'Running'
display(p2)
b=''
x=-1
# translated to 102
with open(r'Your_tranlsated_file_here.txt', 'a', encoding='UTF-8') as fp:
    for i in data:
        x+=1
        p2.value+=1
        # Use debug mode to monitor "x" if you what to continue translation next time.
        # Replace '0' to pervious last status value. 
        if x > 0:
            max_len=len(i.dict()['page_content'])
            res = i.dict()['page_content']
            if max_len > 9:
                if max_len < 463:
                    temp=translation(i.dict()['page_content'], max_length=max_len)
                    res= temp[0]['translation_text']
                else:
                    res, history = model.chat(tokenizer, prompt.format(question=i.dict()['page_content']), history=[])
            b= b + res + '\n'
            fp.write('\n'+ res)
            fp.flush()
    fp.close()